# EDA & Preprocessing

In [43]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
from io import StringIO
np.set_printoptions(precision=4)
import json

In [82]:
import pandas as pd
import numpy as np
import json
from lxml.html import parse
from urllib.request import urlopen
import urllib
from pandas.io.parsers import TextParser
import requests

# 1. Loading/HTML/XML/JSON

## 실습내용 요약
- Reading and writing data in text format
- Reading json data
- Reading XML and HTML with web scraping
- Interacting with HTML and web APIs

## Reading and writing data in text format

### Review
- Dataframe
    - 표 같은 스프레드시트 형식의 자료 구조로 여러 개의 칼럼이 있는데, 각 칼럼은 서로 다른 종류의 값(숫자, 문자열, 불리언 등)을 포함
    - pd.DataFrame() 함수를 사용하여 정의
    - 입력할 데이터는 numpy의 2차원 array, Python 딕셔너리 등의 형태 가능

Pandas 패키지를 활용해 주어진 ex1.csv 파일을 불러온 후, 표시하시오.

In [4]:
ex1 = pd.read_csv('data/ex1.csv')
# ex1 = pd.read_csv('data/ex1.csv', sep=',')
ex1

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Pandas 패키지를 활용해 주어진 ex2.csv 파일을 불러온 후, 표시하시오.

In [5]:
ex2 = pd.read_csv('data/ex2.csv')
ex2

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


첫 줄을 헤더로 활용하지 않고, 주어진 컬럼의 이름(a, b, c, d, message)을 활용하여 불러오시오.

In [6]:
ex2 = pd.read_csv('data/ex2.csv', header=None, names=['a', 'b', 'c', 'd', 'message'])
ex2

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


데이터를 불러올 때, column index를 a와 b의 조합으로 설정하시오.

In [7]:
ex2 = pd.read_csv('data/ex2.csv', header=None, names=['a', 'b', 'c', 'd', 'message'], index_col=['a', 'b'])
ex2

,,c,d,message
a,b,,,
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


## Exercise

주어진 csv_mindex.csv 파일을 불러온 후, 표시하시오. 추가로, 데이터의 column index를 key1과 key2로 설정하시오

#### Solution

In [8]:
df = pd.read_csv('data/csv_mindex.csv', index_col=['key1', 'key2'])
df

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

## Handle exception file formats

ex4.csv에는 1, 2, 4번째 줄에 잘못된 데이터가 존재한다. 해당 행을 스킵하여 데이터를 불러오시오.

In [9]:
ex4 = pd.read_csv('data/ex4.csv')
ex4

,,,,# hey!
a,b,c,d,message
# just wanted to make things more difficult for you,NaN,NaN,NaN,NaN
# who reads csv files with computers,anyway?,NaN,NaN,NaN
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [10]:
ex4 = pd.read_csv('data/ex4.csv', skiprows=[0, 2, 3])
ex4

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Reading json data

### Review
- JSON 형식
    - JavaScript Object Notation라는 의미의 축약어로 데이터를 저장하거나 전송할 때 많이 사용되는 경량의 DATA 교환 형식
    - JSON 표현식은 사람과 기계 모두 이해하기 쉬우며 용량이 작아서, 최근에는 JSON이 XML을 대체해서 데이터 전송 등에 많이 사용

In [11]:
obj="""{
    "name": "Wes",
    "places_lived": ["United States", "Spain", "Germany"],
    "pet": null,
    "siblings": [
        {"name": "Scott", "age": 25, "pet": "Zuko"},
        {"name": "Katie", "age": 33, "pet": "Cisco"}
    ]
}"""

주어진 JSON 형식의 데이터 obj를 임포트하여, 표시하시오. 추가로, key 값을 활용해 데이터에 접근하시오.

In [12]:
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 25, 'pet': 'Zuko'},
  {'name': 'Katie', 'age': 33, 'pet': 'Cisco'}]}

In [13]:
print('Name: ', result['name'])
print('Places lived: ', result['places_lived'])

Name:  Wes
Places lived:  ['United States', 'Spain', 'Germany']


obj.json 파일을 불러와 위와 같이 임포트하시오.

In [14]:
with open('data/obj.json') as f:
    result = json.load(f)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 25, 'pet': 'Zuko'},
  {'name': 'Katie', 'age': 33, 'pet': 'Cisco'}]}

임포트한 json형식 데이터의 자손에 대한 정보를 dataframe 형식으로 변환하시오.

In [15]:
asjson = json.dumps(result)
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,25
1,Katie,33


## Reading XML and HTML with web scraping

### Review
- HTML(HyperText Markup Language: 데이터를 웹상에 표현하기 위해 디자인된 언어
    - HTML을 정해진 태그가 존재하며, 데이터와 그에 대한 표현을 함께 표현
- XML(eXtensible Markup Language): 데이터의 저장하고 전달하기 위해 디자인된 언어
    - XML은 사용자가 정의한 태그 활용 가능

주어진 example.html 파일을 불러와 파싱하고 a태그 중에서 하이퍼링크(href)가 걸려있는 태그를 찾고 링크의 주소와 컨텐츠를 추출하시오.

In [16]:
!cat 'data/example.html'
# Window에서는 !type 'data/example.html'

<html>
<head>
</head>
<body>
This is a test page.<br>
<a href='http://www.postech.ac.kr'>POSTECH</a><br>
<a href='http://ime.postech.ac.kr'>Department of Industrial and Management Engineering</a><br>
<a href='http://aim.postech.ac.kr'>Analytics & Information Management Lab</a><br>
<br>
<br>
<table>
  <tr>
    <td>&nbsp;</td>
    <td>Knocky</td>
    <td>Flor</td>
    <td>Ella</td>
    <td>Juan</td>
  </tr>
  <tr>
    <td>Breed</td>
    <td>Jack Russell</td>
    <td>Poodle</td>
    <td>Streetdog</td>
    <td>Cocker Spaniel</td>
  </tr>
  <tr>
    <td>Age</td>
    <td>16</td>
    <td>9</td>
    <td>10</td>
    <td>5</td>
  </tr>
  <tr>
    <td>Owner</td>
    <td>Mother-in-law</td>
    <td>Me</td>
    <td>Me</td>
    <td>Sister-in-law</td>
  </tr>
  <tr>
    <td>Eating Habits</td>
    <td>Eats everyone's leftovers</td>
    <td>Nibbles at food</td>
    <td>Hearty eater</td>
    <td>Will eat till he explodes</td>
  </tr>
</table>
</body>
</html>

In [17]:
parsed = parse('data/example.html')
doc = parsed.getroot()
links = doc.findall('.//a')
print(links)
for link in links:
    href = link.get('href')
    conts = link.text_content()
    print('하이퍼링크 주소: ', href)
    print('컨텐츠: ', conts)

[<Element a at 0x7fc3a02633b0>, <Element a at 0x7fc3a02630b0>, <Element a at 0x7fc3a0263cb0>]
하이퍼링크 주소:  http://www.postech.ac.kr
컨텐츠:  POSTECH
하이퍼링크 주소:  http://ime.postech.ac.kr
컨텐츠:  Department of Industrial and Management Engineering
하이퍼링크 주소:  http://aim.postech.ac.kr
컨텐츠:  Analytics & Information Management Lab


### Exercise
- 주어진 함수를 활용해 표의 내용을 파싱하여 dataframe으로 변환하시오.

In [32]:
def unpack(row, kind='td'):
    elts = row.findall('.//'.format(kind))
    return [val.text_content() for val in elts]
def parse_table_data(table):
    rows = table.findall('.//tr')
    data = [unpack(r) for r in rows[0:]]
    return TextParser(data).get_chunk()

#### Solution

In [37]:
tables = doc.findall('.//table')
table = tables[0]
call_data = parse_table_data(table)
call_data

,,Knocky,Flor,Ella,Juan
0,Breed,Jack Russell,Poodle,Streetdog,Cocker Spaniel
1,Age,16,9,10,5
2,Owner,Mother-in-law,Me,Me,Sister-in-law
3,Eating Habits,Eats everyone's leftovers,Nibbles at food,Hearty eater,Will eat till he explodes


### Exercise
- 사이트 https://finance.yahoo.com/quote/AAPL/options?ltr=1 을 활용해, 표에 들어있는 애플의 주가를 dataframe으로 변환하시오. (주어진 함수 이용)

In [55]:
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = unpack(rows[0], kind='th')
    data = [unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

#### Solution

In [58]:
parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc = parsed.getroot()
tables = doc.findall('.//table')
tables
calls = tables[0]
puts = tables[1]

In [60]:
call_data = parse_options_data(calls)
put_data = parse_options_data(puts)

In [64]:
call_data.head(3)

,Contract Name,Contract Name.1,Last Trade Date,Last Trade Date.1,Strike,Strike.1,,.1,Last Price,Last Price.1,...,Change,Change.1,% Change,% Change.1,Volume,Volume.1,Open Interest,Open Interest.1,Implied Volatility,Implied Volatility.1
0,AAPL210430C00060000,AAPL210430C00060000,2021-04-22 9:35AM EDT,60.0,60.0,73.08,0.0,0.0,0.0,0.0,...,0.00%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL210430C00065000,AAPL210430C00065000,2021-04-20 3:50PM EDT,65.0,65.0,68.08,0.0,0.0,0.0,0.0,...,0.00%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAPL210430C00070000,AAPL210430C00070000,2021-04-26 9:34AM EDT,70.0,70.0,64.35,0.0,0.0,0.0,0.0,...,0.00%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
put_data.head(3)

,Contract Name,Contract Name.1,Last Trade Date,Last Trade Date.1,Strike,Strike.1,,.1,Last Price,Last Price.1,...,Change,Change.1,% Change,% Change.1,Volume,Volume.1,Open Interest,Open Interest.1,Implied Volatility,Implied Volatility.1
0,AAPL210430P00060000,AAPL210430P00060000,2021-04-23 3:36PM EDT,60.0,60.0,0.02,0.0,0.0,0.0,0.0,...,50.00%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAPL210430P00065000,AAPL210430P00065000,2021-03-30 10:27AM EDT,65.0,65.0,0.03,0.0,0.0,0.0,0.0,...,50.00%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAPL210430P00070000,AAPL210430P00070000,2021-04-21 9:50AM EDT,70.0,70.0,0.01,0.0,0.0,0.0,0.0,...,50.00%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Interacting with HTML and web APIs

주어진 API에 request를 쏘고, 얻어진 데이터를 보이시오. (API_url: https://api.github.com/repos/pydata/pandas/milestones/28/labels)

In [84]:
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
resp = requests.get(url)
data = json.loads(resp.text)
data

[{'id': 76811,
  'node_id': 'MDU6TGFiZWw3NjgxMQ==',
  'url': 'https://api.github.com/repos/pandas-dev/pandas/labels/Bug',
  'name': 'Bug',
  'color': 'e10c02',
  'default': False,
  'description': None},
 {'id': 76812,
  'node_id': 'MDU6TGFiZWw3NjgxMg==',
  'url': 'https://api.github.com/repos/pandas-dev/pandas/labels/Enhancement',
  'name': 'Enhancement',
  'color': '4E9A06',
  'default': False,
  'description': None},
 {'id': 127681,
  'node_id': 'MDU6TGFiZWwxMjc2ODE=',
  'url': 'https://api.github.com/repos/pandas-dev/pandas/labels/Refactor',
  'name': 'Refactor',
  'color': 'FCE94F',
  'default': False,
  'description': 'Internal refactoring of code'},
 {'id': 129350,
  'node_id': 'MDU6TGFiZWwxMjkzNTA=',
  'url': 'https://api.github.com/repos/pandas-dev/pandas/labels/Build',
  'name': 'Build',
  'color': '75507B',
  'default': False,
  'description': 'Library building on various platforms'},
 {'id': 134699,
  'node_id': 'MDU6TGFiZWwxMzQ2OTk=',
  'url': 'https://api.github.com/repos